# Discovering Immediate Mode GUIs

## Intro

Graphical User Interfaces (GUIs) handle how your application is presented to the user.

Most GUI frameworks rely on complex patterns like **MVC**, which separate application logic into layers: Model (the data), View (the presentation), and Controller (the logic). Thus, the code which handles the state of the application is split in different places, which can make it hard to follow, and to synchronize, once the application grows.

**Immediate Mode GUI (IMGUI)** frameworks, like **Dear ImGui**, take a radically different approach: *the code which renders the widgets and the code which handles the user interactions are in the same place*, i.e. in the rendering loop. Handling the state of the application, and adding features, becomes extremely simple.

The example below illustrates this concept, where an application could display a counter, together with a button to increase it:

```python
# on_frame is called at each frame
def on_frame():
    # Display a text widget 
    imgui.text(f"counter={counter}")
    # Display a button, and return true if clicked
    if imgui.button("increase counter"):
        counter += 1  # Perform an action immediately on interaction
```

### Why Use Immediate Mode GUIs?

Here are some key benefits:

1. **Simplicity**: UI code is clean and readable. You describe what to display and interact with, frame by frame.
2. **Minimal State Management**: No complex state updates; you control state directly in your code.
3. **Performance**: Optimized for real-time rendering and interaction.

Widgets are defined and user interactions are managed in the same code section. This immediate approach is particularly useful for:

**Notes:**
- **Separation of concerns**: The immediate mode paradigm is compatible with best practices, such as having a separate Model (or Application State). 
  See the next example for an illustration.
- **Limitations:** The Dear ImGui library is not designed for fully "skinnable" UIs (although custom "themes" or "styles" are available), and will not support complex font rendering (left to right, etc).


## Contents of this tutorial 
In this tutorial, you will:

1. Learn the basics of Immediate Mode GUIs.
2. Explore the fundamental widgets provided by **Dear ImGui**.
3. Use **Hello ImGui** to quickly set up applications with minimal boilerplate code.
4. Build a small, interactive project to solidify your understanding.

Let’s dive in and explore the straightforward yet powerful world of Immediate Mode GUI programming!


## Introducing *Dear ImGui*, *Hello ImGui*, and *Dear ImGui Bundle*

- **[Dear ImGui](https://github.com/ocornut/imgui)**: A lightweight and fast C++ library for Immediate Mode GUI programming, with over 60k stars on GitHub. It enables rapid creation of UI components such as buttons, sliders, and text fields with minimal code, leveraging GPU rendering for exceptional performance.
- **[Hello ImGui](https://pthom.github.io/hello_imgui)**: A powerful C++ wrapper around Dear ImGui designed to streamline apps creation with Dear ImGui, and to simplify complex tasks such as layout handling, FPS idling, and creating mobile and desktop applications. It reduces boilerplate and adds enhanced utilities, making it ideal for both simple prototypes and advanced production-grade GUIs.
- **[Dear ImGui Bundle](https://pthom.github.io/imgui_bundle)**: an extensive set of ready-to-use widgets and libraries, based on ImGui and Hello ImGui. It also provides bindings for Python, enabling you to create GUI applications in **Python**, which we will explore in this tutorial.



## Installing the required Libraries

*In this web tutorial, `imgui-bundle` is pre-installed*. If you are running this tutorial locally, you can install imgui-bundle following the instructions [for Python](https://pthom.github.io/imgui_bundle/install_python.html) or [for C++](https://pthom.github.io/imgui_bundle/install_cpp.html).


---

# Hello Dear ImGui: your first application with Dear ImGui

Let's start by creating a simple application "Hello, World" application, where we will demonstrate several aspects of ImGui and Hello ImGui.

In this application:
- The world is represented by an image of the Earth globe (using `hello_imgui.image_from_asset`)
- The globe size can be adjusted using a slider. This part of the code demonstrates how to handle user editions with ImGui widgets: in Python bindings, widgets return a tuple `(changed, new_value)`, where changed indicates if the widget was interacted with.
- A counter can also be incremented by clicking a button
- The Frame rate is displayed, as a way to demonstrate how Hello ImGui automatically handles application idling.
- An application State class is created, as an illustration of how to separate the application logic from the GUI logic.
- We use Hello ImGui "Run" function to start the application.


In [ ]:
from imgui_bundle import imgui, hello_imgui, ImVec2

# Define a class to hold the application state
class AppState:
    counter = 0
    globe_size = 100.0


# This is our GUI function, which will be called by Hello ImGui at *every* frame:
# It will display the GUI of our application *and* handle user events
def gui(app_state: AppState):
    global counter, globe_size

    # Step 1: Display a label and an image
    # -------------------------------------
    # Display a label saying "Hello, " and an image of the world below
    imgui.text("Hello, ")
    # Will display an image from the "assets/" folder, with a size of (app_state.globe_size, 0) (0 means keep the aspect ratio)
    # (see )
    hello_imgui.image_from_asset("images/world.png", ImVec2(app_state.globe_size, 0))

    # Step 2: Display a slider and handle its value changes
    # -----------------------------------------------------
    # Display a slider to change the globe size
    # Some widgets in ImGui will occupy the full width of the window by default, 
    # so we can use set_next_item_width to limit the width
    imgui.set_next_item_width(hello_imgui.em_size(10))
    # In Python, ImGui widgets often return a tuple of (value_changed: bool, new_value: ...), hence we store
    # Note: this is specific to Python bindings (See next part to see how to handle this in C++)
    _changed, app_state.globe_size = imgui.slider_float("Globe size", app_state.globe_size, 50.0, 400.0)

    # Step 3: Display a counter and a button to increment it
    # ------------------------------------------------------
    imgui.separator_text("Counter")
    imgui.text("Counter: %d" % app_state.counter)
    imgui.same_line()
    if imgui.button("Increment"):
        app_state.counter += 1

    # Step 4: Display the frame rate
    # ------------------------------
    # Display the application frame rate, with an indication if the application is idling:
    # (by default, Hello ImGui will let applications idle after 3 seconds of inactivity)
    imgui.separator_text("Frame rate")
    imgui.text(f"{hello_imgui.frame_rate(): .1f} FPS")
    if hello_imgui.get_runner_params().fps_idling.is_idling:
        imgui.same_line()  # By default, widgets will be placed on the next line, unless we call imgui.same_line() before.
        imgui.text("(Idling)")

    # Step 5: Demonstrate different themes
    # ------------------------------------
    imgui.separator_text("Themes")
    tweaked_theme = hello_imgui.get_runner_params().imgui_window_params.tweaked_theme
    if imgui.button("Cherry"):
        tweaked_theme.theme = hello_imgui.ImGuiTheme_.cherry
        hello_imgui.apply_tweaked_theme(tweaked_theme)
    imgui.same_line()
    if imgui.button("Darcula Darker"):
        tweaked_theme.theme = hello_imgui.ImGuiTheme_.darcula_darker
        hello_imgui.apply_tweaked_theme(tweaked_theme)
    imgui.same_line()
    if imgui.button("White is White"):
        tweaked_theme.theme = hello_imgui.ImGuiTheme_.white_is_white
        hello_imgui.apply_tweaked_theme(tweaked_theme)


    # Step 6: Display a button to exit the application, with a tooltip
    # ----------------------------------------------------------------
    imgui.separator_text("Exit")
    note = """
    - In the case of a desktop application, setting "app_shall_exit" to True will close the window at the next frame.
    - In the case of a web application (or a mobile phone application), this has no effect.
    """ 
    if imgui.button("Exit"):
        hello_imgui.get_runner_params().app_shall_exit = True
    imgui.set_item_tooltip(note)


def main():
    # Create an instance of the application state
    app_state = AppState()
    # Create a parameterless function that calls the GUI function with the application state
    gui_fn = lambda: gui(app_state)
    # And let hello_imgui run our parameterless function
    hello_imgui.run(gui_fn)

if __name__ == "__main__":
    main()

## Hello, Dear ImGui, in C++

As an illustration, below is the C++ code to create this application using Dear ImGui and Hello ImGui.

TODO: add a link to emscripten demo of this app. Explain how modified values are handled by the widgets.

```cpp
#include "imgui.h"
#include "hello_imgui/hello_imgui.h"

int counter = 0;

void Gui() {
   // To be continued...
}

int main() { HelloImGui::Run(Gui); }
```

# Deploy web applications in Python with Dear ImGui Bundle

![Jack](images/jack_tangled_frameworks.jpg)
![Tom](images/tom_tangled_frameworks.webp)

# What's next?

* Explore more widgets within ImGui (buttons, inputs, plots)
* Explore even more widgets withing Dear ImGui Bundle (e.g., plots, 3D guizmo, advanced image viewer, etc.)
* Learn about Hello ImGui: docking layouts (e.g., for advanced UIs).
* ... and much more!